# Part 1: Asset Class and Backtesting

This first part of the report presents the core quantitative research of the project. The analysis begins with a discussion of how trading models are designed and tested in practice, including considerations such as indicator construction, data quality, and signal stability. Attention is then given to the structure of futures markets and how contract-specific features are incorporated into the models. From there, the report builds toward two complete trading systems, each with clearly defined logic and risk management.

This section concludes by evaluating the standalone performance of each model and considering how they might be combined into a portfolio to improve return and risk characteristics.

## 2. Developing Trading Models
This section defines the blueprint used across the project: why a model exists, how its rules are specified, how data is handled, which instruments/markets are included, and how risk, entries/exits, and rebalancing are managed. Detailed implementations for the mean-reversion and trend-following systems follow later.

1. **Model Purpose**

Each model targets a specific, testable market behavior with a clear objective (return profile, risk exposure, role in the broader portfolio). It starts from a hypothesis: what persistent mechanism or inefficiency is being traded, and why should it exist going forward? Rules are derived from this idea—not from arbitrary indicator blends or in-sample “improvements.” In institutional contexts, models are designed to meet portfolio or business constraints (capacity, turnover, margin usage, drawdown tolerances), not to maximize backtest P&L in isolation.

2. **Rules and Variations**

Once the objective of the model is defined, the next step is to translate that into a concise set of trading rules. The goal is to keep the rule set minimal—only what’s essential to express the core market hypothesis. Simpler models are typically more robust and easier to maintain. Any added complexity must be justified by a clear, measurable benefit. Complexity, by default, is a liability.
Every rule should have a clear rationale grounded in market logic—not just an improvement in backtest metrics. If a rule improves simulated returns but doesn’t make sense in the context of the model’s intent, it likely won’t hold up in live trading.

After defining the base rules, it’s reasonable to test a few controlled variations. This is not the same as parameter optimization. For instance, in a mean-reversion setup where price closing 4 standard deviations below a 60-day Bollinger Band tends to revert upward by around 2 standard deviations, testing similar setups (e.g., 3 or 5 deviations, or regression windows of 30 or 90 days) can reveal whether the model is structurally sound.
What should be avoided is exhaustive optimization. Finding that the “optimal” entry is at 3.78 standard deviations on a 73-day regression targeting a 1.54 deviation move is just curve-fitting noise. These kinds of outputs may look statistically compelling insample, but have no real predictive power.
We will stick to sensible ranges and variations that preserve the model’s structure. Avoid overfitting disguised as precision.

3. **Handling Data**

The simplest way to avoid overfitting is to divide our dataset into two parts: one for developing the model and the other for testing it. Rules should only be formulated using the in-sample period. Once the logic is set, it’s tested on out-of-sample data. This helps verify whether the model generalizes beyond the data it was built on.
In more advanced workflows, walk-forward testing or cross-validation can also be used to assess model stability over time. However, for this project, a clean split between in-sample and out-of-sample data provides a practical and transparent structure.

4. **Asset Class**

Asset classes can be defined in different ways. The typical breakdown—equities, fixed income, FX, and commodities—works well for most market participants. But in systematic trading, it’s often more useful to classify markets by the type of instrument traded.

For a quant, the structure and behavior of the instrument—whether it's spot, futures, options, or ETFs—can have more impact on strategy design than the underlying asset itself. Execution, leverage, and data characteristics tend to vary more by instrument type than by the asset class label. In the following chapters, we will discuss the instrument used, futures, and their characteristics.

5. **Investment Universe**

The investment universe defines the set of markets the strategy will trade. This decision has a major impact on performance and risk. Most professional strategies are portfolio-based, not single-market. Designing a strategy around just one asset severely limits its robustness. Even a well-designed model will underperform if the chosen market enters a prolonged weak regime. Diversification across markets increases the chance that at least some instruments are in favorable conditions. A sound approach is to build strategies that can be applied across multiple instruments rather than tailoring them too tightly to one market.


A common pitfall is selecting instruments that have performed well recently—consciously or not—which introduces bias. Selection should instead consider liquidity, volatility structure, and alignment with the strategy logic. Each asset class brings different challenges to universe selection, particularly equities, where survivorship bias, corporate actions, and index membership changes can distort backtests if not handled correctly. However, in this project, by focusing on futures, we will address other equally important issues, which will be analyzed in detail in the following chapters.

6. **Allocation and Risk Level**

Allocation specifies how capital and risk are distributed across positions and models. Here, positions are based on volatility to target a fixed risk per trade/market, supporting a consistent portfolio risk budget.
Throughout this project, the models target risk levels that are consistent with institutional standards. The aim is to deliver meaningful returns while maintaining risk profiles that could be integrated into a professional portfolio. This includes managing exposure, drawdown control, and maintaining capital efficiency across strategies and market regimes.

7. **Entry and Exit Rules**

Entry and exit rules matter, but they’re just one part of a broader framework. In many portfolio-driven strategies, performance often depends more on overall exposure and risk balance than on precise entry timing. A solid model should not rely on perfectly timed trades but instead on consistent logic that aligns with the broader positioning and strategy design.
Specific entry and exit rules for the trend-following and mean-reversion models are discussed in detail in later sections, using combinations of moving averages, risk-adjusted metrics, and volatility filters.

8. **Rebalancing**

Rebalancing is often overlooked but plays a critical role in maintaining consistent risk exposure, especially in medium- to long-term strategies. Rebalancing will be handled at the portfolio level, ensuring consistent risk alignment across strategies.

With the blueprint set, we move on to the tools behind the research. We spell out the backtesting engine, data handling, and the assumptions baked into every run results reflect what could actually be traded. We begin with Zipline Reloaded, outlining how we configure multi-asset futures simulations and why it fits our needs.